# Integrate Malignant cells in multiple patients to and plot FOXA2+ vs FOXA- cells

In [ ]:
import scanpy as sc
import anndata
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import harmonypy
import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.font_manager
from matplotlib import font_manager
from matplotlib.font_manager import fontManager, FontProperties
import infercnvpy as cnv



def setup_dirs(outDir):
    figuresDir = os.path.join(outDir, 'figures')
    dataDir = os.path.join(outDir, 'data')
    tablesDir = os.path.join(outDir, 'tables')
    os.makedirs(figuresDir, exist_ok=True)
    os.makedirs(dataDir, exist_ok=True)
    os.makedirs(tablesDir, exist_ok=True)
    return figuresDir, dataDir, tablesDir

def force_arial():
    arial_font_path = '/home/salehis/projects/cdm/fonts/arial.ttf'
    font_manager.fontManager.addfont(arial_font_path)
    prop = font_manager.FontProperties(fname=arial_font_path)
    print("Arial font forced")

# set the font
def find_arial_font():
    arial_font = None
    for font in font_manager.findSystemFonts():
        #if font.lower().endswith("arial.ttf"):
        if "arial" in font.lower():
            arial_font = font
            break
        if arial_font:
            print("Found Arial font at: ", arial_font)
            prop = font_manager.FontProperties(fname=arial_font)
            sns.set(font=prop.get_name())
    if arial_font is None:
        print("Arial font not found")
        force_arial()


def filter_genes(adata):
    """
    Filtering the following genes to avoid the dominant effect of
    IG{}V, (Immunoglobulin variable)
    TR{}, (T cell receptor variable genes)
    linc, (Long intergenic non-coding),
    genes starting with RP (ribosomal protein),
    genes starting with MT- (mitochondrial genes)
    HLA genes
    """
    genes = [x for x in adata.var.index.tolist() if "MT-" not in x]
    genes = [x for x in genes if "." not in x]
    genes = [x for x in genes if not x.startswith("RP")]
    genes = [x for x in genes if "linc" not in x.lower()]
    genes = [x for x in genes if "TRA" not in x.upper()]
    genes = [x for x in genes if "TRB" not in x.upper()]
    genes = [x for x in genes if "TRG" not in x.upper()]
    genes = [x for x in genes if "TRD" not in x.upper()]
    genes = [x for x in genes if "IGKV" not in x.upper()]
    genes = [x for x in genes if "IGHV" not in x.upper()]
    genes = [x for x in genes if "IGLV" not in x.upper()]
    genes = [x for x in genes if "-" not in x.upper() and "HLA" not in x.upper()]
    adata = adata[:, genes].copy()
    return adata


def add_gene_binary_status(adata, gene, threshold=np.log1p(1), use_counts=False):
    """
    Find a cut-off for expressed vs not expressed
    very simple, expressed, if there is more than 1 count (i.e., log1p > log(1+1) = np.log1p(1))
    add a column to adata.obs, {gene}_is_expressed
    """    
    assert gene in adata.var_names, f"Gene {gene} not in adata.var_names..."
    # drop the column if it exists
    if f'{gene}_is_expressed' in adata.obs.columns:
        adata.obs.drop(f'{gene}_is_expressed', axis=1, inplace=True)
    if f'{gene}_EXPR' in adata.obs.columns:
        adata.obs.drop(f'{gene}_EXPR', axis=1, inplace=True)
    if use_counts:
        adata.obs[f'{gene}_EXPR'] = adata.layers['counts'][:, (adata.var_names == gene)].A
    else:
        adata.obs[f'{gene}_EXPR'] = adata[:, gene].X.A
    adata.obs[f'{gene}_is_expressed'] = adata.obs[f'{gene}_EXPR'] > threshold
    adata.obs[f'{gene}_is_expressed'] = adata.obs[f'{gene}_is_expressed'].astype('category')    
    return adata 

find_arial_font()

In [ ]:
outDir = '/data1/shahs3/users/salehis/sclc/results/rebuttal/nat_comms/integration'
figuresDir, dataDir, tablesDir = setup_dirs(outDir)

sc.settings.figdir = figuresDir
sc.set_figure_params(dpi_save=300, vector_friendly=True)

rsync -azvp --relative \
    iris:/data1/shahs3/users/salehis/sclc/./results//rebuttal/nat_comms/integration/figures/*.p* \
    /Users/salehis/Projects/sclc/rebuttal_code/SCLC_MET/

In [ ]:
adata_path = '/data1/shahs3/users/salehis/sclc/results/patient_met/foxa2_umaps_19/data/rna_19_2K_harmony.h5ad'
adata = sc.read_h5ad(adata_path)

In [ ]:
# Do one without any batch correction 

In [ ]:
# 

In [ ]:
## Test that the old result is there

main_genes = ['FOXA2']
for i, gene in enumerate(main_genes):    
    adata.obs[f'{gene}_is_expressed_str'] = adata.obs[f'{gene}_is_expressed'].astype(str)
    sc.tl.embedding_density(adata, basis='umap', groupby=f'{gene}_is_expressed_str')
    sc.pl.embedding_density(adata, basis='umap', key=f'umap_density_{gene}_is_expressed_str', save=f"{gene}_expr_umap_density.pdf")


## Recompute Harmony with many more iterations

In [ ]:
max_iter_harmony = 100
harmony_column = 'sample'
sc.external.pp.harmony_integrate(adata, key=harmony_column, max_iter_harmony=max_iter_harmony)
sc.pp.neighbors(adata, use_rep='X_pca_harmony')
sc.tl.umap(adata)
sc.pl.umap(adata, color=harmony_column, save=f're_harmony_max_iter_{max_iter_harmony}.pdf')


In [ ]:
def mini_process(adata, use_harmony=False, do_scale=False, max_iter_harmony=100, harmony_column='sample'):
    """
    Minimal Processing on the top 2000 genes. 

    1. Normalize the data
    2. Log1p the data
    3. Scale the data
    4. Run PCA
    5. Run Harmony
    6. Run UMAP
    """
    adata.X = adata.layers['counts'].copy()
    assert adata.X.max() > 100, "The data is not raw counts"
    assert adata.X.min() >= 0, "The data is not raw counts"
    # Normalize the data
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    if do_scale:
        sc.pp.scale(adata, max_value=10)
    sc.tl.pca(adata, svd_solver='arpack')
    use_rep = 'X_pca'
    if use_harmony:
        sc.external.pp.harmony_integrate(adata, key=harmony_column, max_iter_harmony=max_iter_harmony)
        use_rep = 'X_pca_harmony'
    sc.pp.neighbors(adata, use_rep=use_rep)
    sc.tl.umap(adata)
    return adata

for use_harmony in [False, True]:
    for do_scale in [False, True]:
        conf_str = f"use_harmony_{use_harmony}_do_scale_{do_scale}"
        adata = sc.read_h5ad(adata_path)
        adata = mini_process(adata, use_harmony=use_harmony, do_scale=do_scale)
        sc.pl.umap(adata, color=harmony_column, save=f're_harmony_max_iter_{max_iter_harmony}_{conf_str}.pdf')
        adata.write(os.path.join(dataDir, f'adata_{conf_str}.h5ad'))


# Now plot the heat embedding
for use_harmony in [False, True]:
    for do_scale in [False, True]:
        conf_str = f"use_harmony_{use_harmony}_do_scale_{do_scale}"
        adata = sc.read_h5ad(os.path.join(dataDir, f'adata_{conf_str}.h5ad'))
        main_genes = ['FOXA2']
        for i, gene in enumerate(main_genes):    
            adata = add_gene_binary_status(adata, 'FOXA2', threshold=0, use_counts=True)
            adata.obs[f'{gene}_is_expressed_str'] = adata.obs[f'{gene}_is_expressed'].astype(str)
            sc.tl.embedding_density(adata, basis='umap', groupby=f'{gene}_is_expressed_str')
            sc.pl.embedding_density(adata, basis='umap', key=f'umap_density_{gene}_is_expressed_str', save=f"{gene}_expr_umap_density_{conf_str}.pdf")


In [ ]:

# UMAP of patients
sc.pl.umap(adata, color='patient_id', save='_umap_patients_19.pdf', ncols=1, title='Patient ID')
sc.pl.umap(adata, color='sample', save='_umap_patients_19_sample.pdf', ncols=1, title='Patient ID')

# Violin plot of FOXA2 expression per patient
adata.obs['FOXA2_expr_log'] = adata[:, 'FOXA2'].X.A.flatten()
order = adata.obs.groupby('sample')['FOXA2_expr_log'].mean().sort_values(ascending=False).index.tolist()
plt.clf()
fig, ax = plt.subplots(figsize=(10, 5))
sc.pl.violin(adata, groupby='sample', keys='FOXA2', order=order, rotation=90, ax=ax)
plt.savefig(os.path.join(figuresDir, 'foxa2_violin_per_patient.pdf'), bbox_inches='tight')
plt.close()


main_genes = ['FOXA2']
for i, gene in enumerate(main_genes):    
    adata.obs[f'{gene}_is_expressed_str'] = adata.obs[f'{gene}_is_expressed'].astype(str)
    sc.tl.embedding_density(adata, basis='umap', groupby=f'{gene}_is_expressed_str')
    sc.pl.embedding_density(adata, basis='umap', key=f'umap_density_{gene}_is_expressed_str', save=f"{gene}_expr_umap_density.pdf")

In [ ]:

sc.pl.embedding_density(adata, basis='umap', key='umap_density_FOXA2_is_expressed', save=f"foxa2_expr_umap_density_with_harmony_false_blue.png", group='False', color_map='Blues')